In [1]:
from google.colab import drive
import os
# Install p7zip-full for handling .7z archives
!apt-get install -y p7zip-full
!pip install nibabel nilearn dicom2nifti
from scipy.ndimage import shift

from nilearn.image import clean_img, resample_to_img
import nibabel as nib
import os
import dicom2nifti
import nibabel as nib
from nilearn.image import clean_img, resample_to_img, smooth_img, resample_img
from nilearn.masking import compute_brain_mask
from nilearn.datasets import load_mni152_template
import numpy as np
from nilearn.decomposition import CanICA
from sklearn.preprocessing import StandardScaler
from nilearn.connectome import ConnectivityMeasure
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import pandas as pd
import glob
!pip install scikit-image
from skimage.registration import phase_cross_correlation



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-8).
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 29.1 MB/s eta 0:00:00


In [2]:
# Mount Google Drive
drive.mount('/content/drive')

# Define paths
uploaded_file_path = '/content/drive/MyDrive/schizconnect_COBRE_images_22498.7z.001'
output_dir = '/content/extracted_data'
os.makedirs(output_dir, exist_ok=True)

print("Google Drive mounted and paths defined.")


Mounted at /content/drive
Google Drive mounted and paths defined.


In [3]:
# Rename the file to .7z
renamed_file_path = uploaded_file_path.replace('.7z.001', '.7z')
os.rename(uploaded_file_path, renamed_file_path)

print(f"File renamed to: {renamed_file_path}")


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/schizconnect_COBRE_images_22498.7z.001' -> '/content/drive/MyDrive/schizconnect_COBRE_images_22498.7z'

In [4]:
# Extract the renamed .7z file
!7z x "/content/drive/MyDrive/schizconnect_COBRE_images_22498.7z" -o"/content/extracted_data"


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/                                 1 file, 2277077170 bytes (2172 MiB)

Extracting archive: /content/drive/MyDrive/schizconnect_COBRE_images_22498.7z
--
Path = /content/drive/MyDrive/schizconnect_COBRE_images_22498.7z
Type = 7z
Physical Size = 2277077170
Headers Size = 13601
Method = LZMA2:24
Solid = +
Blocks = 2

  0%      0% 308 - COBRE/sub-A00000368/ses-20110 . 20110101_task-rest_bold.nii.gz                                                                           2% 329 - COBRE/sub-A00000541/ses-201001 . 0000

In [5]:
# List the extracted files
'''for root, dirs, files in os.walk(output_dir):
    for file in files:
        print(os.path.join(root, file))'''

save_dir = '/content/fmri/corrected_nifti_data/'
os.makedirs(save_dir, exist_ok=True)

process_dir = '/content/fmri/processed_imaging_data/'
os.makedirs(process_dir, exist_ok=True)

spatial_maps_dir = '/content/fmri/spatial_maps_data/'
os.makedirs(spatial_maps_dir, exist_ok=True)
ica = CanICA(n_components=20, mask_strategy='background')
scaler = StandardScaler()

In [7]:
import os
import glob
import nibabel as nib
import numpy as np
from nilearn.image import resample_img, resample_to_img, smooth_img, clean_img
from nilearn.masking import compute_brain_mask
from nilearn.input_data import NiftiMasker
from nilearn.datasets import load_mni152_template
from skimage.registration import phase_cross_correlation
from scipy.ndimage import shift
from sklearn.preprocessing import StandardScaler
from joblib import Parallel, delayed
from nilearn.maskers import NiftiMasker
from nilearn.decomposition import CanICA

# Initialize scaler globally for reuse
scaler = StandardScaler()

# Motion Correction Function
def motion_correction(fmri_data, reference_volume=None):
    if reference_volume is None:
        reference_volume = np.mean(fmri_data[:5], axis=0)  # Use average of first 5 volumes as reference

    corrected_images = [reference_volume]
    reference_volume /= np.max(reference_volume)

    for i in range(1, fmri_data.shape[0]):
        fmri_data[i] /= np.max(fmri_data[i])
        fmri_data[i] = np.nan_to_num(fmri_data[i])
        shift_values, _, _ = phase_cross_correlation(reference_volume, fmri_data[i])
        corrected_image = shift(fmri_data[i], shift_values)
        corrected_images.append(corrected_image)

    corrected_data = np.stack(corrected_images, axis=0)
    return corrected_data

# Feature Extraction Function
def extract_features(nifti_img, save_dir, process_dir, i):
    #print(f"Processing file {i + 1}/{len(nifti_files)}: {nifti_img}")
    fmri_data = nifti_img.get_fdata()

    # Step 1: Motion Correction
    corrected_data = motion_correction(fmri_data)
    corrected_data[corrected_data < 0] = 0  # Set negative values to zero
    corrected_nifti_img = nib.Nifti1Image(corrected_data, nifti_img.affine)


    # Load the MNI template
    mni_template = load_mni152_template()
    downsampled_template = resample_img(
        mni_template,
        target_affine=corrected_nifti_img.affine,
        target_shape=corrected_nifti_img.shape[:3],
        interpolation="linear",
        force_resample=True,
        copy_header=True,
    )

    # Resample to template
    normalized_img = resample_to_img(corrected_nifti_img, downsampled_template, interpolation="linear", force_resample=True, copy_header=True)

    #print("Min:", np.min(normalized_img.get_fdata()), "Max:", np.max(normalized_img.get_fdata()), "Mean:", np.mean(normalized_img.get_fdata()))
    # Compute brain mask and resample
    #brain_mask = compute_brain_mask(normalized_img, lower_cutoff=0.3, upper_cutoff=0.8)
    brain_mask = mni_template.get_fdata() > 0  # Threshold the template to create a binary mask
    brain_mask_img = nib.Nifti1Image(brain_mask.astype(np.uint8), mni_template.affine)


    resampled_mask = resample_to_img(brain_mask_img, normalized_img, interpolation="nearest", force_resample=True, copy_header=True)

    # Clean the image
    cleaned_img = clean_img(
        normalized_img,
        detrend=True,
        standardize=True,
        low_pass=0.2,
        high_pass=0.005,
        t_r=1.0,
        mask_img=resampled_mask,
        n_jobs=-1,
    )

    #print("Min:", np.min(cleaned_img.get_fdata()), "Max:", np.max(cleaned_img.get_fdata()))
    # Smooth the image
    smoothed_img = smooth_img(cleaned_img, fwhm=6)
    print(smoothed_img.shape)
    fmri_4d_data = smoothed_img.get_fdata()  # Shape: (time_points, x_dim, y_dim, z_dim)
    print(fmri_4d_data.shape)

    # Compute the total number of voxels (x_dim * y_dim * z_dim)

    # Create a NiftiMasker
    num_voxels = np.prod(fmri_4d_data.shape[:3])  # Spatial dimensions (x, y, z)
    print(num_voxels)
    masker = NiftiMasker(standardize=True)

    # Fit the masker to your cleaned and smoothed 4D NIfTI image
    masker.fit(smoothed_img)
    #print("BYE", time_series_data.shape)

    data_2d = masker.transform(smoothed_img)  # Shape: (time_points, voxels)
    # Initialize ICA
    ica = CanICA(
        n_components=50,         # Number of components
        random_state=0,         # For reproducibility
        high_pass=0.24999970197677612,
        memory="nilearn_cache",  # Use same cache
        t_r=2.0,
        mask=masker.mask_img_  # Use the mask from NiftiMasker
    )



    # Fit ICA with the 4D NIfTI image
    ica.fit(smoothed_img)

    # Get the spatial components
    components = ica.components_

    # Spatial maps and feature extraction
    feature_vectors = []
    # Save the spatial components as individual NIfTI files

    mask = masker.mask_img_
    mask_data = mask.get_fdata()
    component_full = np.zeros(smoothed_img.shape[:3])
    for j, component in enumerate(components):
        # Create a NIfTI image for each component
        # Reshape the 1D component back into 3D space (matching the fmri data shape)
        #print("component shape", component.size)

        voxel_indices = np.where(mask_data != 0)  # This gives the coordinates (i, j, k) of non-zero mask voxels

        # Place the component values into the correct 3D positions according to the mask
        component_full[voxel_indices] = component  # Assign the component values to the mask positions
        # Define a threshold dynamically based on data
        threshold = np.mean(component_full) + np.std(component_full)

        # Create binary mask
        binary_mask = (component_full > threshold).astype(int)
        binary_mask = binary_mask.astype(np.int32)  # Convert to int32


        # Generate spatial map
        spatial_map = nib.Nifti1Image(binary_mask, smoothed_img.affine)

        masker = NiftiMasker(mask_img=spatial_map, standardize=True)
        try:
          # Debugging: Check before applying the masker
          print("Checking masker...")
          mask_d = spatial_map.get_fdata()
          if np.count_nonzero(mask_d) == 0:
              raise ValueError("Computed mask is empty. Adjust the mask computation.")

          print("Checking smoothed image...")
          img_data = smoothed_img.get_fdata()
          if np.isnan(img_data).sum() > 0:
              print("Warning: NaN values found in smoothed image!")
              img_data = np.nan_to_num(img_data)
          if np.count_nonzero(img_data) == 0:
              raise ValueError("Smoothed image is empty. Verify preprocessing steps.")

          # Apply transformation
          print("Applying masker transformation...")
          time_series = masker.fit_transform(smoothed_img)
          print("Time series data shape:", time_series.shape)

        except Exception as e:
          print("Error during masker transformation:", e)

        #time_series = masker.fit_transform(smoothed_img)

        # Normalize time series
        time_series = scaler.fit_transform(time_series)

        # Compute voxel features
        voxel_means = np.mean(time_series, axis=0)
        voxel_stds = np.std(time_series, axis=0)

        # Combine into feature vector
        features = np.concatenate([voxel_means, voxel_stds])
        feature_vectors.append(features)

    return feature_vectors

# Main Processing Loop
def process_nifti_files(nifti_files, save_dir, process_dir):
    '''results = Parallel(n_jobs=-1)(
        delayed(extract_features)(nib.load(nii_file), save_dir, process_dir, i)
        for i, nii_file in enumerate(nifti_files)
    )'''
    results = []
    for i, nii_file in enumerate(nifti_files):
      if i<=30:
        print(f"Processing file {i + 1}/{len(nifti_files)}: {nii_file}")
        results.append(extract_features(nib.load(nii_file), save_dir, process_dir, i))
        #for i, nii_file in enumerate(nifti_files)'''
    return results

    return results
nifti_file_path = '/content/extracted_data/COBRE/sub*/ses*/func/*.nii.gz'

# Glob all files
nifti_files = glob.glob(nifti_file_path)
# Debugging step to print the files being processed
def is_empty_nifti(file_path):
    img = nib.load(file_path)
    return np.count_nonzero(img.get_fdata()) == 0  # Check if data is empty

nifti_files = glob.glob(nifti_file_path)
nifti_files = [f for f in nifti_files if not is_empty_nifti(f)]
def is_valid_nifti(file_path):
    try:
        nib.load(file_path)
        return True
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return False

nifti_files = glob.glob(nifti_file_path)
nifti_files = [f for f in nifti_files if is_valid_nifti(f)]

print("NIfTI files found:", nifti_files)

# Run the pipeline
feature_vectors = process_nifti_files(nifti_files, save_dir, process_dir)

print("Preprocessing complete!")


NIfTI files found: ['/content/extracted_data/COBRE/sub-A00028405/ses-20120101/func/sub-A00028405_ses-20120101_task-rest_bold.nii.gz', '/content/extracted_data/COBRE/sub-A00014590/ses-20110101/func/sub-A00014590_ses-20110101_task-rest_bold.nii.gz', '/content/extracted_data/COBRE/sub-A00014830/ses-20090101/func/sub-A00014830_ses-20090101_task-rest_bold.nii.gz', '/content/extracted_data/COBRE/sub-A00002480/ses-20110101/func/sub-A00002480_ses-20110101_task-rest_bold.nii.gz', '/content/extracted_data/COBRE/sub-A00001452/ses-20100101/func/sub-A00001452_ses-20100101_task-rest_bold.nii.gz', '/content/extracted_data/COBRE/sub-A00028404/ses-20110101/func/sub-A00028404_ses-20110101_task-rest_bold.nii.gz', '/content/extracted_data/COBRE/sub-A00024959/ses-20090101/func/sub-A00024959_ses-20090101_task-rest_bold.nii.gz', '/content/extracted_data/COBRE/sub-A00015648/ses-20110101/func/sub-A00015648_ses-20110101_task-rest_bold.nii.gz', '/content/extracted_data/COBRE/sub-A00024953/ses-20090101/func/sub-A

/usr/local/lib/python3.11/dist-packages/nilearn/decomposition/_base.py:492: UserWarning: memory_level is currently set to 0 but a Memory object has been provided. Setting memory_level to 1.
  self.masker_.fit()
/usr/local/lib/python3.11/dist-packages/nilearn/decomposition/canica.py:292: UserWarning: memory_level is currently set to 0 but a Memory object has been provided. Setting memory_level to 1.
  components = _MultiPCA._raw_fit(self, data)
<ipython-input-7-ad76adb94955>:115: FutureWarning: The nifti_maps_masker_ attribute is deprecated andwill be removed in Nilearn 0.11.3. Please use maps_masker_ instead.
  ica.fit(smoothed_img)


Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 428)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 391)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 384)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 366)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 412)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 435)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 445)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 425)
Checking masker...
Checking smoothed image...
Applying masker transformation...


/usr/local/lib/python3.11/dist-packages/nilearn/decomposition/_base.py:492: UserWarning: memory_level is currently set to 0 but a Memory object has been provided. Setting memory_level to 1.
  self.masker_.fit()
/usr/local/lib/python3.11/dist-packages/nilearn/decomposition/canica.py:292: UserWarning: memory_level is currently set to 0 but a Memory object has been provided. Setting memory_level to 1.
  components = _MultiPCA._raw_fit(self, data)
<ipython-input-7-ad76adb94955>:115: FutureWarning: The nifti_maps_masker_ attribute is deprecated andwill be removed in Nilearn 0.11.3. Please use maps_masker_ instead.
  ica.fit(smoothed_img)


Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 490)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 443)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 468)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 472)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 513)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 448)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 517)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 498)
Checking masker...
Checking smoothed image...
Applying masker transformation...


...

Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 469)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 408)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 87)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 455)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 374)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 464)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 519)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 313)
Checking masker...
Checking smoothed image...
Applying masker transformation...
T

/usr/local/lib/python3.11/dist-packages/nilearn/decomposition/_base.py:492: UserWarning: memory_level is currently set to 0 but a Memory object has been provided. Setting memory_level to 1.
  self.masker_.fit()
/usr/local/lib/python3.11/dist-packages/nilearn/decomposition/canica.py:292: UserWarning: memory_level is currently set to 0 but a Memory object has been provided. Setting memory_level to 1.
  components = _MultiPCA._raw_fit(self, data)
<ipython-input-7-ad76adb94955>:115: FutureWarning: The nifti_maps_masker_ attribute is deprecated andwill be removed in Nilearn 0.11.3. Please use maps_masker_ instead.
  ica.fit(smoothed_img)


Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 457)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 98)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 446)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 430)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 477)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 478)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 413)
Checking masker...
Checking smoothed image...
Applying masker transformation...
Time series data shape: (150, 328)
Checking masker...
Checking smoothed image...
Applying masker transformation...
T

/usr/local/lib/python3.11/dist-packages/nilearn/decomposition/_base.py:492: UserWarning: memory_level is currently set to 0 but a Memory object has been provided. Setting memory_level to 1.
  self.masker_.fit()
/usr/local/lib/python3.11/dist-packages/nilearn/decomposition/canica.py:292: UserWarning: memory_level is currently set to 0 but a Memory object has been provided. Setting memory_level to 1.
  components = _MultiPCA._raw_fit(self, data)


KeyboardInterrupt: 

In [8]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.cluster import KMeans

# ... (Your previous code) ...

# Ensure all feature vectors are NumPy arrays and have the same length
#feature_vectors = [np.array(f) for f in feature_vectors]

# Find the maximum feature vector length
max_length = max(len(sublist) for f in feature_vectors for sublist in f)  # Modified to find max length across all sublists

# Pad all vectors to the same length
X = np.array([[np.pad(sublist, (0, max_length - len(sublist)), mode='constant')
              for sublist in f] for f in feature_vectors])  # Pad each sublist

print("Final feature matrix shape:", X.shape)  # Should be (num_samples, max_length)

# ... (Rest of your code) ...

NameError: name 'feature_vectors' is not defined

In [38]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.cluster import KMeans


# Define Autoencoder class
class Autoencoder(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, latent_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.ReLU(),
            nn.Linear(128, input_dim)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

# Initialize Autoencoder
print(X.shape, X.shape[0], X.shape[1])
input_dim = X.shape[2]  # Ensure correct shape after padding
latent_dim = 32
autoencoder = Autoencoder(input_dim, latent_dim)

# Set up optimizer and loss function
optimizer = optim.Adam(autoencoder.parameters(), lr=1e-3)
criterion = nn.MSELoss()

# Convert X to a PyTorch tensor
X_tensor = torch.FloatTensor(X)

# Train Autoencoder
num_epochs = 100
for epoch in range(num_epochs):
    autoencoder.train()
    optimizer.zero_grad()
    encoded, decoded = autoencoder(X_tensor)
    loss = criterion(decoded, X_tensor)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")

# Extract latent features
autoencoder.eval()
with torch.no_grad():
    fmri_features, _ = autoencoder(X_tensor)

# Convert features to NumPy for clustering
fmri_features_np = fmri_features.numpy()
print('dim', fmri_features_np.shape)
# Apply K-Means Clustering
n_clusters = 2  # Choose the number of clusters
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
# Reshape to ensure it's (num_samples, latent_dim)
fmri_features_np = fmri_features_np.reshape(fmri_features_np.shape[0], -1)
cluster_labels = kmeans.fit_predict(fmri_features_np)

print("Cluster Labels:", cluster_labels)


(22, 50, 2586) 22 50
Epoch [10/100], Loss: 0.1099
Epoch [20/100], Loss: 0.0437
Epoch [30/100], Loss: 0.0304
Epoch [40/100], Loss: 0.0215
Epoch [50/100], Loss: 0.0165
Epoch [60/100], Loss: 0.0140
Epoch [70/100], Loss: 0.0123
Epoch [80/100], Loss: 0.0109
Epoch [90/100], Loss: 0.0098
Epoch [100/100], Loss: 0.0089
dim (22, 50, 32)
Cluster Labels: [0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 1 0 0 0 1 0]


In [4]:
import zipfile
# Mount Google Drive
drive.mount('/content/drive')
# Define paths
uploaded_file_path = '/content/drive/MyDrive/dataverse_files.zip'
data_dir = '/content/extracted_eeg_data'
os.makedirs(data_dir, exist_ok=True)


# Extract the contents of the zip file
with zipfile.ZipFile(uploaded_file_path, 'r') as zip_ref:
    zip_ref.extractall(data_dir)
print("Google Drive mounted and paths defined.")

Mounted at /content/drive
Google Drive mounted and paths defined.


In [5]:
import os
import numpy as np
!pip install mne
from mne.io import read_raw_edf
from mne import Epochs, pick_types
from mne.time_frequency import psd_array_welch

# Directory containing .edf files
output_dir = "/content/eeg_features/"

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Feature extraction function
def extract_features(raw, output_file, label):
    # Select EEG channels only
    raw.pick_types(eeg=True)

    # Preprocessing: Filter data (e.g., bandpass filter between 0.5-50 Hz)
    raw.filter(0.5, 50., fir_design='firwin')

    # Divide raw data into epochs (e.g., 2-second windows)
    events = np.array([[i, 0, 1] for i in range(0, int(raw.n_times), int(raw.info['sfreq'] * 2))])
    event_id = {'Epoch': 1}
    epochs = Epochs(raw, events, event_id, tmin=0, tmax=2, baseline=None, preload=True)

    # Compute power spectral density (PSD) for each epoch
    psds, freqs = psd_array_welch(
        epochs.get_data(),  # Raw data as a NumPy array
        sfreq=raw.info['sfreq'],  # Sampling frequency
        fmin=0.5,  # Minimum frequency
        fmax=50.,  # Maximum frequency
        n_fft=256  # FFT size
    )

    # Compute band-specific power features (delta, theta, alpha, beta, gamma)
    bands = {
        "delta": (0.5, 4),
        "theta": (4, 8),
        "alpha": (8, 12),
        "beta": (12, 30),
        "gamma": (30, 50)
    }
    features = []
    for band, (low, high) in bands.items():
        band_mask = (freqs >= low) & (freqs < high)  # Create mask for the band
        if np.sum(band_mask) == 0:  # Ensure the mask has valid frequencies
            raise ValueError(f"No frequencies found in the band {band}: ({low}-{high} Hz)")
        band_power = psds[:, :, band_mask].mean(axis=-1)  # Mean power for the band
        features.append(band_power)

    # Concatenate all features into a single feature vector
    features = np.hstack(features)  # Concatenate features into a 2D array
    labels = np.full((features.shape[0], 1), label)  # Shape: (num_epochs, 1)

    # Concatenate labels with features
    labeled_features = np.hstack([features, labels])  # Shape: (num_epochs, num_features + 1)

    print(f"Shape of features: {features.shape}, Label: {label}, Final shape: {labeled_features.shape}")

    # Save features to a file
    try:
        np.save(output_file, labeled_features)
        print(f"Saved features to {output_file}")
    except Exception as e:
        print(f"Failed to save features for file {file_name}: {e}")
# Loop through all .edf files in the directory
for file_name in os.listdir(data_dir):
    if file_name.endswith(".edf"):
        file_path = os.path.join(data_dir, file_name)
        print(f"Processing file: {file_path}")

        try:
            # Load the EDF file
            raw = read_raw_edf(file_path, preload=True)
            # Extract label from the filename (custom logic based on your dataset)
            if file_name.startswith('h'):
                label = 0  # Healthy
            elif file_name.startswith('s'):
                label = 1  # Sick
            else:
                print(f"Skipping file {file_name} as no label mapping is defined.")
                continue  # Skip files with unknown labels
            print(f"Assigned label {label} for file {file_name}")
            # Extract features and save to output directory
            output_file = os.path.join(output_dir, f"{os.path.splitext(file_name)[0]}_features.npy")
            extract_features(raw, output_file, label=label)

        except Exception as e:
            print(f"Error processing file {file_name}: {e}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 66.8 MB/s eta 0:00:00
Processing file: /content/extracted_eeg_data/s05.edf
Extracting EDF parameters from /content/extracted_eeg_data/s05.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 222499  =      0.000 ...   889.996 secs...
Assigned label 1 for file s05.edf
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 1651 samp

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.3s


1 bad epochs dropped
Effective window size : 1.024 (s)
Shape of features: (444, 95), Label: 1, Final shape: (444, 96)
Saved features to /content/eeg_features/s05_features.npy
Processing file: /content/extracted_eeg_data/s03.edf
Extracting EDF parameters from /content/extracted_eeg_data/s03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 240999  =      0.000 ...   963.996 secs...
Assigned label 1 for file s03.edf
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 50.00 Hz
- 

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.3s


Shape of features: (481, 95), Label: 1, Final shape: (481, 96)
Saved features to /content/eeg_features/s03_features.npy
Processing file: /content/extracted_eeg_data/h08.edf
Extracting EDF parameters from /content/extracted_eeg_data/h08.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 227499  =      0.000 ...   909.996 secs...
Assigned label 0 for file h08.edf
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff freq

...

Shape of features: (454, 95), Label: 0, Final shape: (454, 96)
Saved features to /content/eeg_features/h07_features.npy
Processing file: /content/extracted_eeg_data/h04.edf
Extracting EDF parameters from /content/extracted_eeg_data/h04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 231249  =      0.000 ...   924.996 secs...
Assigned label 0 for file h04.edf
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff freq

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


Shape of features: (462, 95), Label: 0, Final shape: (462, 96)
Saved features to /content/eeg_features/h04_features.npy
Processing file: /content/extracted_eeg_data/s02.edf
Extracting EDF parameters from /content/extracted_eeg_data/s02.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 286249  =      0.000 ...  1144.996 secs...
Assigned label 1 for file s02.edf
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff freq

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


Shape of features: (572, 95), Label: 1, Final shape: (572, 96)
Saved features to /content/eeg_features/s02_features.npy


In [32]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
import os

# Load features from saved .npy files
def load_features(data_dir):
    feature_list = []
    labels_list = []

    # Ensure to use the correct directory variable (output_dir)
    for file_name in os.listdir(data_dir):
        if file_name.endswith("_features.npy"):  # Make sure we are looking at the correct files
            file_path = os.path.join(data_dir, file_name)
            try:
                features = np.load(file_path)
                feature_list.append(features[:, :-1])  # Exclude labels (everything except the last column)
                labels_list.append(features[:, -1])   # Labels are the last column (single column)

                print(f"Loaded {file_name}: features shape {features[:, :-1].shape}, labels shape {features[:, -1].shape}")

            except Exception as e:
                print(f"Error loading {file_name}: {e}")

    # Concatenate all features and labels from all files
    X = np.vstack(feature_list)  # Feature matrix (all rows, all feature columns)
    y = np.hstack(labels_list)   # Label vector (all labels)

    # Print the shapes of the final feature matrix and label vector
    print(f"Final shape of features matrix (X): {X.shape}")
    print(f"Final shape of labels vector (y): {y.shape}")

    return X, y


# Directory containing EEG feature files
output_dir = "/content/eeg_features/"
X, y = load_features(output_dir)

# Split into training and testing sets
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from torch.utils.data import DataLoader, TensorDataset

# Assuming X and y are already loaded correctly
# X: Features matrix (shape: 14411, 95)
# y: Labels vector (shape: 14411,)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
X_train = ss.fit_transform(X_train)  # Fit & transform on train
X_test = ss.transform(X_test)

def add_noise(X, noise_level=0.01):
    noise = noise_level * np.random.randn(*X.shape)
    return X + noise

X_train_noisy = add_noise(X_train, noise_level=0.02)
X_train_combined = np.vstack((X_train, X_train_noisy))
y_train_combined = np.hstack((y_train, y_train))  # Duplicate labels

# Convert to tensors again
X_train_combined = torch.tensor(X_train_combined, dtype=torch.float32)
y_train_combined = torch.tensor(y_train_combined, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

# Create DataLoader for batching and shuffling
train_data = TensorDataset(X_train_combined, y_train_combined)
test_data = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

# Define the model
class EEGModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(EEGModel, self).__init__()
        self.eeg_branch = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.5),  # Dropout for regularization
            nn.Linear(256, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)  # Output layer for classification (2 classes)
        )

    def forward(self, x):
        return self.eeg_branch(x)

# Initialize the model
model = EEGModel(input_dim=X.shape[1], hidden_dim=256, output_dim=2)  # Adjust hidden_dim as needed

# Define loss function and optimizer
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.utils.class_weight import compute_class_weight

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = torch.tensor(class_weights, dtype=torch.float32)

# Update the loss function with class weights
criterion = nn.CrossEntropyLoss(weight=class_weights)

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop (use the same code you already have)
# Training loop (use the same code you already have)
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Add learning rate scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5, verbose=True)

# Modify training loop
best_loss = float('inf')
for epoch in range(100):
    model.train()
    running_loss = 0.0

    for batch in train_loader:
        inputs, labels = batch
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}, Loss: {avg_loss}")

    # Check for early stopping
    if avg_loss < best_loss:
        best_loss = avg_loss
        torch.save(model.state_dict(), 'best_model.pth')  # Save best model
    else:
        print("No improvement, applying LR scheduler")
        scheduler.step(avg_loss)  # Reduce learning rate if loss stops improving


# Evaluate the model
import torch
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score

# Ensure model is in evaluation mode
model.eval()

# Store predictions and ground truth labels
y_true = []
y_pred = []
y_probs = []  # For probability scores

# Disable gradient computation for evaluation
with torch.no_grad():
    for batch in test_loader:
        inputs, labels = batch
        outputs = model(inputs)  # Get raw logits
        probs = torch.softmax(outputs, dim=1)[:, 1]  # Convert to probabilities (class 1)
        _, predicted = torch.max(outputs, 1)  # Get class with highest score

        y_true.extend(labels.numpy())  # Actual labels
        y_pred.extend(predicted.numpy())  # Predicted labels
        y_probs.extend(probs.numpy())  # Probability scores

# Convert lists to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)
y_probs = np.array(y_probs)

# Compute evaluation metrics
accuracy = accuracy_score(y_true, y_pred)
classification_rep = classification_report(y_true, y_pred)
conf_matrix = confusion_matrix(y_true, y_pred)
roc_auc = roc_auc_score(y_true, y_probs)
TN, FP, FN, TP = confusion_matrix(y_true, y_pred).ravel()

print('True Positive(TP)  = ', TP)
print('False Positive(FP) = ', FP)
print('True Negative(TN)  = ', TN)
print('False Negative(FN) = ', FN)
# Print results
print(f"✅ Accuracy: {accuracy:.4f}")
print("\n🔹 Classification Report:\n", classification_rep)
print("\n🔹 Confusion Matrix:\n", conf_matrix)
print(f"\n🔹 ROC-AUC Score: {roc_auc:.4f}")
accuracy =  (TP + TN) / (TP + FP + TN + FN)
from sklearn.metrics import f1_score

f1 = f1_score(y_true, y_pred)
print(f"🔹 F1 Score: {f1:.4f}")
print('Accuracy of the binary classifier = {:0.3f}'.format(accuracy))




Loaded h11_features.npy: features shape (457, 95), labels shape (457,)
Loaded h13_features.npy: features shape (482, 95), labels shape (482,)
Loaded h05_features.npy: features shape (472, 95), labels shape (472,)
Loaded h10_features.npy: features shape (557, 95), labels shape (557,)
Loaded s07_features.npy: features shape (672, 95), labels shape (672,)
Loaded s02_features.npy: features shape (572, 95), labels shape (572,)
Loaded h12_features.npy: features shape (449, 95), labels shape (449,)
Loaded s08_features.npy: features shape (455, 95), labels shape (455,)
Loaded h07_features.npy: features shape (454, 95), labels shape (454,)
Loaded h04_features.npy: features shape (462, 95), labels shape (462,)
Loaded h06_features.npy: features shape (464, 95), labels shape (464,)
Loaded s05_features.npy: features shape (444, 95), labels shape (444,)
Loaded h08_features.npy: features shape (454, 95), labels shape (454,)
Loaded s11_features.npy: features shape (679, 95), labels shape (679,)
Loaded

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1, Loss: 0.6193532003806188
Epoch 2, Loss: 0.5349007833462495
Epoch 3, Loss: 0.5091159825141613
Epoch 4, Loss: 0.49207958781860284
Epoch 5, Loss: 0.4810545054765848
Epoch 6, Loss: 0.4688686350217232
Epoch 7, Loss: 0.4689480081288772
No improvement, applying LR scheduler
Epoch 8, Loss: 0.44927043666148325
Epoch 9, Loss: 0.4427904313837988
Epoch 10, Loss: 0.7227860208622803
No improvement, applying LR scheduler
Epoch 11, Loss: 0.42948948517000884
Epoch 12, Loss: 0.4222867842873878
Epoch 13, Loss: 0.4180391402198718
Epoch 14, Loss: 0.41050276169057426
Epoch 15, Loss: 0.403953189327872
Epoch 16, Loss: 0.3984912736616896
Epoch 17, Loss: 0.39416903526296276
Epoch 18, Loss: 0.40121010183935335
No improvement, applying LR scheduler
Epoch 19, Loss: 0.38396604685388375
Epoch 20, Loss: 0.3820242869871608
Epoch 21, Loss: 0.3742683664316962
Epoch 22, Loss: 0.37123640498818733
Epoch 23, Loss: 0.3672219093382006
Epoch 24, Loss: 0.3677579497797249
No improvement, applying LR scheduler
Epoch 25, 

In [49]:
class EEGFeatureExtractor(nn.Module):
    def __init__(self, input_dim, hidden_dim, feature_dim):
        super(EEGFeatureExtractor, self).__init__()
        self.eeg_branch = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, hidden_dim),
            nn.ReLU()
        )
        self.feature_layer = nn.Linear(hidden_dim, feature_dim)  # Feature vector instead of classification

    def forward(self, x):
        x = self.eeg_branch(x)  # Extract intermediate features
        return self.feature_layer(x)  # Output feature representation

In [53]:
eeg_model = EEGFeatureExtractor(input_dim=95, hidden_dim=256, feature_dim=64)
pretrained_dict = torch.load('best_model.pth')  # Load saved model
model_dict = eeg_model.state_dict()

# Remove layers that have mismatched sizes (like the first Linear layer)
pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict and v.shape == model_dict[k].shape}

# Update model weights
model_dict.update(pretrained_dict)
eeg_model.load_state_dict(model_dict, strict=False)  # strict=False allows skipping mismatched layers

print("✅ Loaded pretrained weights, excluding mismatched layers.")


# Freeze the EEG model (optional)
eeg_model.eval()  # Set to evaluation mode to avoid updating weights during training


✅ Loaded pretrained weights, excluding mismatched layers.


<ipython-input-53-1b33c9f0595d>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pretrained_dict = torch.load('best_model.pth')  # Load saved model


EEGFeatureExtractor(
  (eeg_branch): Sequential(
    (0): Linear(in_features=95, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=256, out_features=256, bias=True)
    (4): ReLU()
  )
  (feature_layer): Linear(in_features=256, out_features=64, bias=True)
)

In [118]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FusionModel(nn.Module):
    def __init__(self, eeg_model, fmri_dim, fusion_dim, output_dim=2):
        super(FusionModel, self).__init__()

        # Use the pretrained EEG feature extractor
        self.eeg_branch = eeg_model

        # fMRI feature extraction (assuming it's 128-dimensional features from autoencoder)
        self.fmri_branch = nn.Linear(fmri_dim, 256)

        # Fusion layer (combining EEG and fMRI features)
        self.fusion_layer = nn.Sequential(
            nn.Linear(320, fusion_dim),  # EEG (64) + fMRI (64) → Fusion
            nn.BatchNorm1d(50),
            nn.ReLU(),
            nn.Linear(fusion_dim, output_dim) # Final classification layer (2 classes)
        )

    def forward(self, eeg, fmri):
        # Extract EEG features (using pretrained EEG model)
        with torch.no_grad():
            eeg_features = self.eeg_branch(eeg)  # Extract EEG embeddings

        # Extract fMRI features
        fmri_features = self.fmri_branch(fmri)  # Extract fMRI embeddings

        # Fix mismatch in dimensions and align EEG and fMRI feature sizes
        eeg_features = eeg_features.unsqueeze(1)  # Add extra dimension to EEG features
        eeg_features = eeg_features.expand(-1, fmri_features.shape[1], -1)  # Match sequence length of fMRI features

        # Fix batch size mismatch if needed
        if fmri_features.shape[0] < eeg_features.shape[0]:
            fmri_features = fmri_features.repeat((eeg_features.shape[0] // fmri_features.shape[0] + 1), 1, 1)
            fmri_features = fmri_features[:eeg_features.shape[0]]  # Trim excess

        # Combine EEG and fMRI features
        combined_features = torch.cat((eeg_features, fmri_features), dim=2)  # Concatenate along the feature dimension

        # Pass through fusion layer for final classification
        output = self.fusion_layer(combined_features)  # Final output (logits for 2 classes)

        return output


In [ ]:
fusion_model = FusionModel(eeg_model=eeg_model, fmri_dim=32, fusion_dim=256, output_dim=2)

optimizer = optim.Adam(fusion_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

print(y_train_combined.shape)


for epoch in range(50):
    optimizer.zero_grad()

    outputs = fusion_model(X_train_combined, fmri_features[:len(X_train_combined)])
    # Ensure model output has the shape [batch_size, 2] (logits)
    print(f"Output shape: {outputs.shape}")  # This should be [batch_size, 2]

    # Ensure target labels are 1D [batch_size]
    print(f"Target shape: {y_train.shape}")  # This should be [batch_size]
    # Assuming the final time step's output is the relevant prediction
    outputs = outputs[:, -1, :]  # Get the last time step (shape: [batch_size, 2])

    # Now, outputs shape will be [batch_size, 2] (which is expected by CrossEntropyLoss)

    loss = criterion(outputs, y_train_combined)

    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item()}")


torch.Size([21616])


In [101]:
import torch
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, f1_score
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F

# Load EEG and fMRI features (assuming they have been loaded already)
# X_test, y_test (EEG features and labels) are already in the previous code
# fmri_features: This is assumed to be preloaded in a similar manner to EEG features.

# Ensure model is in evaluation mode
fusion_model.eval()

# Initialize lists to store predictions and ground truth
y_true = []
y_pred = []
y_probs = []  # For probability scores

# Inference loop
with torch.no_grad():
    for i in range(0, len(X_test), 64):  # Iterate in batches of 64
        # Slice EEG batch
        eeg_batch = X_test[i:i+64]

        # Slice fMRI batch to match EEG batch size
        fmri_batch = fmri_features[i:i+len(eeg_batch)]  # Ensure the batch size matches

        # Ensure fMRI batch is not empty
        if fmri_batch.shape[0] == 0:
            continue  # Skip empty batches

        # Perform inference with the fusion model
        outputs = fusion_model(eeg_batch, fmri_batch)  # Pass both EEG and fMRI data to the model

        # Apply sigmoid activation to the output logits (convert to probabilities)
        probs = torch.sigmoid(outputs)  # Apply sigmoid to get probabilities (shape: [batch_size, 2])

        # Get predicted class based on sigmoid output (threshold at 0.5)
        predicted = (probs[:, 1] > 0.5).long()  # If probability of class 1 is greater than 0.5, predict class 1, else class 0

        # Store results
        y_true.extend(y_test[i:i+len(eeg_batch)].cpu().numpy())  # Actual labels
        y_pred.extend(predicted.cpu().numpy())  # Predicted labels
        y_probs.extend(probs[:, 1].cpu().numpy())  # Probabilities for ROC-AUC (only for class 1)

# Convert lists to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)
y_probs = np.array(y_probs)

# Check for empty arrays
if len(y_true) == 0 or len(y_pred) == 0:
    print("Error: y_true or y_pred is empty. Please check your inference loop.")
else:
    # Ensure that y_true and y_pred are both 1D arrays with binary labels (0 or 1)
    if y_true.ndim > 1:
        y_true = y_true.flatten()
    if y_pred.ndim > 1:
        y_pred = y_pred.flatten()

    # Compute evaluation metrics
    accuracy = accuracy_score(y_true, y_pred)
    classification_rep = classification_report(y_true, y_pred)
    conf_matrix = confusion_matrix(y_true, y_pred)
    roc_auc = roc_auc_score(y_true, y_probs)

    # Confusion matrix values
    TN, FP, FN, TP = confusion_matrix(y_true, y_pred).ravel()

    print('True Positive (TP) =', TP)
    print('False Positive (FP) =', FP)
    print('True Negative (TN) =', TN)
    print('False Negative (FN) =', FN)

    # Print results
    print(f"✅ Accuracy: {accuracy:.4f}")
    print("\n🔹 Classification Report:\n", classification_rep)
    print("\n🔹 Confusion Matrix:\n", conf_matrix)
    print(f"\n🔹 ROC-AUC Score: {roc_auc:.4f}")

    # Calculate final accuracy
    accuracy = (TP + TN) / (TP + FP + TN + FN)
    print(f"Accuracy of the binary classifier = {accuracy:.4f}")

    # Calculate F1 Score
    f1 = f1_score(y_true, y_pred)
    print(f"🔹 F1 Score: {f1:.4f}")


EEG Features Shape: torch.Size([64, 50, 64])
fMRI Features Shape: torch.Size([22, 50, 64])
64
64


ValueError: Classification metrics can't handle a mix of binary and multiclass-multioutput targets